In [1]:
%tensorflow_version 2.x

In [2]:
! pip uninstall -y -q kaggle

In [3]:
pip install --upgrade -q kaggle

     |████████████████████████████████| 61kB 5.4MB/s 


In [4]:
! mkdir -p /root/.kaggle
! mv kaggle.json /root/.kaggle
import kaggle

In [5]:
!kaggle competitions download -c dog-breed-identification

100% 689M/691M [00:08<00:00, 44.2MB/s]
100% 691M/691M [00:08<00:00, 82.8MB/s]


In [6]:
! unzip -q dog-breed-identification.zip

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
labels = pd.read_csv("labels.csv")

In [8]:
labels.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [9]:
import os

In [10]:
os.listdir("train")[:10]

['94af603ff6f83dbc8079a1d315981f5a.jpg',
 'b28dd4ec5ad70c3901a54447ef7ec5ab.jpg',
 'e86a88391d017cc3807cc9653a4a7290.jpg',
 'fab313f7c055d30e79a8590ab05d54f3.jpg',
 '7645e3e336ea56a3d2acdd258190893a.jpg',
 'c7d8a86998c9970e0d398192a016eefe.jpg',
 '435d09bc47562ab35704c2a10441b562.jpg',
 '6c370fc5e1b1840570406446d438cec3.jpg',
 'a326caecceeed4705ba786f1d8504430.jpg',
 '70685fef6eaf5cddd4de563cd6b5763c.jpg']

In [11]:
labels["id"] = labels["id"] + ".jpg"

In [12]:
labels.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07.jpg,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97.jpg,dingo
2,001cdf01b096e06d78e9e5112d419397.jpg,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d.jpg,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62.jpg,golden_retriever


In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
idg = ImageDataGenerator(preprocessing_function=preprocess_input, horizontal_flip=True, validation_split=0.2)

In [14]:
train_idg = idg.flow_from_dataframe(labels, "train", x_col="id", y_col="breed", subset="training", target_size=(224, 224))
val_idg = idg.flow_from_dataframe(labels, "train", x_col="id", y_col="breed", subset="validation", target_size=(224, 224))

Found 8178 validated image filenames belonging to 120 classes.
Found 2044 validated image filenames belonging to 120 classes.


In [15]:
from tensorflow.keras.applications import VGG16

In [16]:
tf_model = VGG16(include_top=True)

553467904/553467096 [==============================] - 5s 0us/step


In [17]:
tf_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [18]:
for layer in tf_model.layers:
  layer.trainable = False
  print(layer, layer.trainable)

<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7fb8e341d048> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fb8e35f7828> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fb8e35f46a0> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7fb8e35f6ba8> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fb8b06ffa90> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fb8a8f91e10> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7fb8a7f3c630> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fb8a7f40588> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fb8a7f40be0> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fb8a7f47a20> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7fb8a7f4fd68> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 

In [19]:
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import SGD, Adagrad, RMSprop
from tensorflow.keras.activations import softmax
from tensorflow.keras.models import Model

In [20]:
x = tf_model.layers[-2].output
x = BatchNormalization(trainable=False) (x)
x = Dropout(0.5) (x)
x = Dense(120, activation="softmax") (x)

In [21]:
final_model = Model(inputs=tf_model.input, outputs=x)
final_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [22]:
final_model.compile(SGD(learning_rate=0.001), categorical_crossentropy, ["acc"])

In [23]:
final_model.fit_generator(train_idg, validation_data=val_idg, epochs=10)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
256/256 [==============================] - 61s 205ms/step - loss: 6.9663 - acc: 0.0387 - val_loss: 2.5260 - val_acc: 0.3894
Epoch 2/10
256/256 [==============================] - 50s 196ms/step - loss: 3.5567 - acc: 0.2513 - val_loss: 1.6652 - val_acc: 0.5548
Epoch 3/10
256/256 [==============================] - 51s 198ms/step - loss: 2.5695 - acc: 0.4007 - val_loss: 1.3702 - val_acc: 0.6159
Epoch 4/10
256/256 [==============================] - 51s 199ms/step - loss: 2.0869 - acc: 0.4900 - val_loss: 1.2463 - val_acc: 0.6477
Epoch 5/10
256/256 [==============================] - 51s 200ms/step - loss: 1.7952 - acc: 0.5383 - val_loss: 1.1766 - val_acc: 0.6580
Epoch 6/10
256/256 [==============================] - 51s 200ms/step - loss: 1.6800 - acc: 0.5624 - val_loss: 1.1205 - val_acc: 0.6722
Epoch 7/10
256/256 [==============================] - 51s 200ms/step - loss: 1.5945 - acc: 0.5898 - val_loss: 1.0919 - val_acc: 0.6879
Epoch 8/10
256/256 [==============================] - 5

In [24]:
! wget https://s3.amazonaws.com/cdn-origin-etr.akc.org/wp-content/uploads/2017/11/20113314/Carolina-Dog-standing-outdoors.jpg

--2020-12-29 08:58:22--  https://s3.amazonaws.com/cdn-origin-etr.akc.org/wp-content/uploads/2017/11/20113314/Carolina-Dog-standing-outdoors.jpg
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.68.110
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.68.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 59481 (58K) [image/jpeg]
Saving to: ‘Carolina-Dog-standing-outdoors.jpg’

Carolina-Dog-standi 100%[===================>]  58.09K  --.-KB/s    in 0.1s    

2020-12-29 08:58:22 (458 KB/s) - ‘Carolina-Dog-standing-outdoors.jpg’ saved [59481/59481]



In [29]:
import cv2
import numpy as np 

In [36]:
! wget https://static.toiimg.com/thumb/msid-60132235,imgsize-169468,width-800,height-600,resizemode-75/60132235.jpg

--2020-12-29 09:09:20--  https://static.toiimg.com/thumb/msid-60132235,imgsize-169468,width-800,height-600,resizemode-75/60132235.jpg
Resolving static.toiimg.com (static.toiimg.com)... 184.86.193.96, 2600:1409:a:38f::216f, 2600:1409:a:389::216f
Connecting to static.toiimg.com (static.toiimg.com)|184.86.193.96|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38667 (38K) [image/jpeg]
Saving to: ‘60132235.jpg’

60132235.jpg        100%[===================>]  37.76K  --.-KB/s    in 0.007s  

2020-12-29 09:09:20 (5.62 MB/s) - ‘60132235.jpg’ saved [38667/38667]



In [44]:
def _predict(img):
  test_image = cv2.imread(img)
  test_image = cv2.resize(test_image, (224, 224))
  test_image = preprocess_input(test_image)
  test_image = np.expand_dims(test_image, axis=0)
  result = np.argmax(final_model.predict(test_image))
  rs_td = labels['breed'][result]
  return rs_td

In [45]:
_predict('60132235.jpg')

'walker_hound'

In [46]:
_predict('Carolina-Dog-standing-outdoors.jpg')

'giant_schnauzer'